In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
torch.manual_seed(42)  # for reproducibility

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Resuming download from 42991616 bytes (29123230 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/zalando-research/fashionmnist?dataset_version_number=4 (42991616/72114846) bytes left.


100%|██████████| 68.8M/68.8M [00:09<00:00, 3.00MB/s]

Extracting files...


Path to dataset files: /home/darshan39/.cache/kagglehub/datasets/zalando-research/fashionmnist/versions/4


In [5]:
df = pd.read_csv(f"{path}/fashion-mnist_train.csv")

In [6]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
x_train, x_test, y_train, y_test = train_test_split(
    df.drop("label", axis=1).values,
    df["label"].values,
    test_size=0.2,
    random_state=42,
)